In [4]:
from google.colab import drive

In [5]:
drive.mount("/content/drive")

Mounted at /content/drive


LOAD THE DATASET

In [1]:
dataset_location = "/content/drive/MyDrive/data/spam.csv"

IMPORT THE LIBRARY

In [2]:
import pandas as pd
import nltk
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.translate.ribes_score import word_rank_alignment
from numpy.lib.shape_base import split
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import LSTM,Dense,Dropout,Input,Embedding,Activation,Flatten
from keras.models import Model
import nltk

READ DATASET AND PREPROCESSING

In [6]:
data = pd.read_csv(dataset_location,encoding = "ISO-8859-1")

In [7]:
data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis = 1,inplace = True)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
nltk.download('stopwords',quiet=True)
nltk.download('all',quiet=True)

True

In [9]:
ps = PorterStemmer()
input = []

In [10]:
for i in range(0,5572):
  v2 = data['v2'][i]

  #removing punctuation
  v2 = re.sub('[^a-zA-Z]',' ',v2)

  #converting to lower case
  v2 = v2.lower()

  #splitting the sentence
  v2 = v2.split()

  #removing the stopwords and stemming
  v2 = [ps.stem(word) for word in v2 if not word in set(stopwords.words('english'))]

  v2 = ' '.join(v2)

  input.append(v2)


In [11]:
#creating document term matrix
cv = CountVectorizer(max_features=2000)
x = cv.fit_transform(input).toarray()
x.shape

(5572, 2000)

In [12]:
le = preprocessing.LabelEncoder()

data['v1'] = le.fit_transform(data['v1'])
data['v1'].unique()

array([0, 1])

In [14]:
y = data['v1'].values

In [16]:
y = y.reshape(-1,1)

In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.4)

MODEL BUILDING - ADDING LAYERS, COMPILING MODEL AND SAVING MODEL


In [18]:
model = Sequential()

In [19]:
model.add(Dense(1565,activation = "relu"))
model.add(Dense(3000,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))
model.add(Flatten())

In [20]:
model.compile(optimizer = "adam",loss = "binary_crossentropy", metrics = ["accuracy"])

In [21]:
model.fit(x_train,y_train,epochs = 15)

Epoch 1/15
105/105 [==============================] - 10s 77ms/step - loss: 0.1497 - accuracy: 0.9548
Epoch 2/15
105/105 [==============================] - 8s 79ms/step - loss: 0.0167 - accuracy: 0.9958
Epoch 3/15
105/105 [==============================] - 9s 81ms/step - loss: 0.0046 - accuracy: 0.9985
Epoch 4/15
105/105 [==============================] - 8s 78ms/step - loss: 0.0025 - accuracy: 0.9997
Epoch 5/15
105/105 [==============================] - 13s 119ms/step - loss: 0.0020 - accuracy: 0.9997
Epoch 6/15
105/105 [==============================] - 8s 71ms/step - loss: 0.0019 - accuracy: 0.9997
Epoch 7/15
105/105 [==============================] - 8s 81ms/step - loss: 0.0018 - accuracy: 0.9997
Epoch 8/15
105/105 [==============================] - 8s 72ms/step - loss: 0.0017 - accuracy: 0.9997
Epoch 9/15
105/105 [==============================] - 9s 86ms/step - loss: 0.0017 - accuracy: 0.9997
Epoch 10/15
105/105 [==============================] - 7s 71ms/step - loss: 0.0015 - acc

In [22]:
model.save("spam-message-classifier.h5")

TESTING THE MODEL

In [23]:
ham = "im donee. come pick me up"
spam = "WINNER$$$$ SMS REPLY 'WIN'"
message = re.sub('[^a-zA-Z]',' ',spam)
message

'WINNER     SMS REPLY  WIN '

TESTING WITH SPAM MESSAGE

In [ ]:
message = message.split()
message = [ps.stem(word) for word in message if not word in set(stopwords.words('english')) ]
message = ' '.join(message)

In [24]:
message1 = cv.transform([message])
message1

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [25]:
TruePredction = model.predict(message1.astype(float))

1/1 [==============================] - 0s 188ms/step


In [26]:
TruePredction > 0.5

array([[False]])

TESTING WITH NORMAL MESSAGE

In [27]:
msg = re.sub('[^a-zA-Z]',' ',ham)
msg

'im donee  come pick me up'

In [28]:
cv.transform([msg])

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [29]:
FalsePredection = model.predict(cv.transform([msg]))

1/1 [==============================] - 0s 89ms/step


In [30]:
FalsePredection > 0.5

array([[False]])